# Chapter 1：FAISS 入门与环境搭建
## 4 第一个FAISS 示例

In [12]:
"""
FAISS基础示例：随机向量精确检索（IndexFlatL2）
流程：生成数据 → 创建索引 → 添加向量 → 执行检索 → 解析结果
"""
import faiss
import numpy as np

# -------------------------- 1. 配置参数与生成数据 --------------------------
# 向量配置
dimension = 128  # 向量维度（模拟图像/文本特征向量）
db_size = 10000  # 数据库向量数量（1万个）
query_size = 5  # 查询向量数量（5个）
k = 10  # 每个查询返回Top-10相似结果

# 生成数据库向量（float32类型，形状：(db_size, dimension)）
np.random.seed(42)  # 固定随机种子，保证结果可复现
db_vectors = np.random.random((db_size, dimension)).astype('float32')

# 生成查询向量（形状：(query_size, dimension)）
query_vectors = np.random.random((query_size, dimension)).astype('float32')


In [13]:
# -------------------------- 2. 创建索引 --------------------------
# 使用IndexFlatL2（基于L2距离的精确检索索引）
# 构造函数参数：向量维度
index = faiss.IndexFlatL2(dimension)

# 查看索引状态（是否已训练，FAISS中精确索引无需训练，默认is_trained=True）
print("索引是否已训练：", index.is_trained)  # 输出：True
print("索引初始向量数量：", index.ntotal)  # 输出：0（未添加向量）


索引是否已训练： True
索引初始向量数量： 0


In [14]:
db_vectors

array([[0.37454012, 0.9507143 , 0.7319939 , ..., 0.42710778, 0.81801474,
        0.8607306 ],
       [0.00695213, 0.5107473 , 0.417411  , ..., 0.316922  , 0.16949275,
        0.55680126],
       [0.9361548 , 0.6960298 , 0.57006115, ..., 0.6967372 , 0.62894285,
        0.87747204],
       ...,
       [0.24655657, 0.30857334, 0.63998616, ..., 0.84720623, 0.54780084,
        0.33188137],
       [0.48411956, 0.17165671, 0.28399438, ..., 0.5098137 , 0.03604472,
        0.22185549],
       [0.70243716, 0.05219886, 0.29882595, ..., 0.8015673 , 0.7206239 ,
        0.947437  ]], shape=(10000, 128), dtype=float32)

In [15]:
print(len(db_vectors))

10000


In [16]:
print(len(db_vectors[0]))


128


In [17]:
# -------------------------- 3. 向索引添加向量 --------------------------
# 使用add()方法添加数据库向量
index.add(db_vectors)

# 查看添加后的索引状态
print("添加后索引向量数量：", index.ntotal)  # 输出：10000

添加后索引向量数量： 10000


In [18]:
# -------------------------- 4. 执行检索 --------------------------
# 使用search()方法执行检索，参数：查询向量、返回结果数
distances, indices = index.search(query_vectors, k)

In [19]:
# -------------------------- 5. 解析检索结果 --------------------------
print("\n" + "="*50)
print("检索结果解析（共{}个查询，每个返回Top-{}）".format(query_size, k))
print("="*50)

for i in range(query_size):
    print("\n查询向量{}:".format(i+1))
    print("  最相似的{}个向量ID：{}".format(k, indices[i]))
    print("  对应的L2距离：{}".format(np.round(distances[i], 4)))  # 保留4位小数



检索结果解析（共5个查询，每个返回Top-10）

查询向量1:
  最相似的10个向量ID：[8769 9385   82 5125 9571 3491 6267 3948 4436 1056]
  对应的L2距离：[13.347  14.5484 14.7083 14.7561 14.8371 14.9125 15.2174 15.3307 15.3792
 15.4888]

查询向量2:
  最相似的10个向量ID：[3314 7078 2779 2931 8916  957 6954 6241  701 8861]
  对应的L2距离：[12.774  13.3655 13.5662 13.6255 13.7859 13.9385 14.058  14.0603 14.0841
 14.0886]

查询向量3:
  最相似的10个向量ID：[3439 3014 4097 6304 7784 1416 3924 5784 2755 9813]
  对应的L2距离：[14.6441 15.4164 15.6935 15.7264 15.771  15.8201 15.8201 15.824  15.8351
 15.8458]

查询向量4:
  最相似的10个向量ID：[8808 9253 5930 7506 8190 5043 3455 7757 1846 6434]
  对应的L2距离：[12.6725 13.0228 13.4774 13.6416 13.8688 13.8751 14.1737 14.1983 14.2814
 14.352 ]

查询向量5:
  最相似的10个向量ID：[7086 7630   59 6994 6033 7375 4870 6823 9959 9235]
  对应的L2距离：[13.6781 13.7864 13.9151 14.1091 14.195  14.2928 14.3377 14.3619 14.4115
 14.5707]


In [20]:
# -------------------------- 6. 索引的其他常用操作 --------------------------
# 1. 保存索引到磁盘
faiss.write_index(index, "faiss_flatl2_index.index")
print("\n索引已保存到：faiss_flatl2_index.index")


索引已保存到：faiss_flatl2_index.index


In [21]:
# 2. 从磁盘加载索引
loaded_index = faiss.read_index("faiss_flatl2_index.index")
print("加载的索引向量数量：", loaded_index.ntotal)  # 输出：10000


加载的索引向量数量： 10000


In [22]:
# 3. 清空索引
loaded_index.reset()
print("清空后索引向量数量：", loaded_index.ntotal)  # 输出：0

清空后索引向量数量： 0
